In [46]:
from random import *
import random

class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width, empty):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        
        #si empty vaut False, aucune cellule n’a de voisines
        if empty == False:
            self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}
        #si empty vaut True, chaque cellule a pour voisines celles qui lui sont contigües dans la grille
        else:
            liste = []
            res = ""
            for i in range(height):
                for j in range(width):
                    
                    #cordoonnées des voisins s'ils existent
                    hautG = i - 1, j - 1
                    haut = i - 1, j
                    hautD = i - 1, j + 1
                    G = i, j - 1
                    D = i, j + 1
                    basG = i + 1, j - 1
                    bas = i + 1, j
                    basD = i + 1, j + 1
                    
                    #les coordonnées des voisins possibles dépendent de la position en hauteur i et en largeur j
                    if i == 0 and j == 0:
                        coordVoisins = {D, bas, basD}
                        liste.append(coordVoisins)
                    elif i == 0 and j == width - 1:
                        coordVoisins = {G, basG, bas}
                        liste.append(coordVoisins)
                    elif i == height - 1 and j == 0:
                        coordVoisins = {hautD, haut, D}
                        liste.append(coordVoisins)
                    elif i == height - 1 and j == width - 1:
                        coordVoisins = {hautG, haut, G}
                        liste.append(coordVoisins)
                    elif j == width - 1:
                        coordVoisins = {hautG, haut, G, basG, bas}
                        liste.append(coordVoisins)
                    elif i == height - 1:
                        coordVoisins = {hautD, haut, hautG, G, D}
                        liste.append(coordVoisins)
                    elif i == 0:
                        coordVoisins = {G, D, basG, basD, bas}
                        liste.append(coordVoisins)
                    elif j == 0:
                        coordVoisins = {haut, hautD, D, bas, basD}
                        liste.append(coordVoisins)
                    else: 
                        coordVoisins = {hautD, haut, hautG, G, D, basD, bas, basG}
                        liste.append(coordVoisins)

            self.neighbors = {}
            var = 0
            for i in range(height):
                for j in range(width):
                    self.neighbors[(i,j)] = liste[var]
                    var += 1
                    
    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def get_cells(self):
        """
        Enumère toutes les cellules du labyrinthe.
        Retour :
            liste (list) : cellules de la grille du labyrinthe
        """
        listeCellules = []
        for i in range(self.height):
            for j in range(self.width):
                listeCellules.append((i, j))
        return listeCellules
                
    def add_wall(self, c1, c2):
        """
        Ajoute un mur entre les deux cellules en paramètres.
        """
        self.neighbors[c1].remove(c2)
        self.neighbors[c2].remove(c1)
          
    def remove_wall(self, c1, c2):
        """
        Supprime un mur entre les deux cellules en paramètres.
        """
        self.neighbors[c1].add(c2)
        self.neighbors[c2].add(c1)
          
    def get_walls(self):
        """
        Enumère tous les murs du labyrinthe.
        Retour : 
            liste (list) : liste de tous les murs sous la forme d’une liste de tuple de cellules
        """
        #liste de toutes les cellules du labyrinthes
        listeCellules = self.get_cells()
        #liste des cellules qui ont un mur
        listeMur = []

        for i in listeCellules:
            for j in listeCellules:
                if i not in self.neighbors[j] and j not in self.neighbors[i] and j in self.get_contiguous_cells(i) :
                    listeMur.append([i,j])
        return listeMur
    
    def fill(self):
        """
        Ajoute tous les murs possibles dans le labyrinthe.
        """
        #liste de toutes les cellules du labyrinthes
        listeCellules = self.get_cells()

        for i in listeCellules:
            for j in listeCellules:
                if j in self.neighbors[i]:
                    self.add_wall(i, j)

    def empty(self):
        """
        Ajoute tous les murs dans le labyrinthe.
        """
        #liste de toutes les cellules du labyrinthes
        listeCellules = self.get_cells()

        for i in listeCellules:
            for j in listeCellules:
                self.neighbors[i].add(j)
                self.neighbors[j].add(i)

    def get_contiguous_cells(self, c):
        """
        Enumère toutes les cellules contigües à la cellule c dans le labyrinthe.
        Retour :
            Liste (list) : liste des cellules contigües à c
        """
        liste = []
        if c[0]-1 >= 0:
            liste.append((c[0]-1, c[1]))
        if  c[0]+1 < self.height:
            liste.append((c[0]+1, c[1]))
        if c[1]-1 >= 0:
            liste.append((c[0], c[1]-1))
        if c[1]+1 < self.width:
            liste.append((c[0], c[1]+1))
        return liste
    
    def get_reachable_cells(self, c):
        """
        Enumère les cellules contiguës à c qui sont dans le voisinage de c dans le labyrinthe.
        Retour :
            Liste (list) : liste des cellules accessibles depuis c
        """
        #liste des cellules contiguës
        listeContigues = []
        #liste liste des cellules contiguës et qui sont voisines à c
        contiguesVoisines = []
        if c[0]-1 >= 0:
            listeContigues.append((c[0]-1, c[1]))
        if  c[0]+1 > c[0]:
            listeContigues.append((c[0]+1, c[1]))
        if c[1]-1 >= 0:
            listeContigues.append((c[0], c[1]-1))
        if c[1]+1 > c[1]:
            listeContigues.append((c[0], c[1]+1))
        for elmt in listeContigues:
            if elmt in self.neighbors[c]:
                contiguesVoisines.append(elmt)
        return contiguesVoisines
    
    @classmethod 
    def gen_btree(cls,h,w):
        """
        génere un labirynthe en supprimant aléatoirement le mur EST ou le mur SUD
        """
        laby = cls(h, w, empty=False)
        lst_cell=laby.get_cells()
        lst_adj_cell=None
        
        for i in lst_cell:
            lst_nonbreakable_cell=laby.get_reachable_cells(i)
            if i[0]+1>=h:
                lst_nonbreakable_cell.append((i[0]+1,i[1]))
            if i[1]+1>=w:
                lst_nonbreakable_cell.append((i[0],i[1]+1))
            
                
            if (i[0]+1,i[1]) in lst_nonbreakable_cell and (i[0],i[1]+1) not in lst_nonbreakable_cell:
                laby.remove_wall(i,(i[0],i[1]+1))
            elif (i[0]+1,i[1]) not in lst_nonbreakable_cell and (i[0],i[1]+1)  in lst_nonbreakable_cell:
                laby.remove_wall(i,(i[0]+1,i[1]))
            elif (i[0]+1,i[1]) in lst_nonbreakable_cell and (i[0],i[1]+1)  in lst_nonbreakable_cell:
                pass
            else:
                tmp=random.randint(0,1)
                if tmp==1:
                    laby.remove_wall(i,(i[0],i[1]+1))
                else:
                    laby.remove_wall(i,(i[0]+1,i[1]))
                    
        return laby
    
    
    
    @classmethod
    def gen_sidewinder(cls, h, w):
        """
        génere un labirynthe via la méthode sidewinder
        """
        #Initialisation : création d’un labyrinthe plein
        laby = cls(h, w, empty=False)
        #Pour i allant de 0 à hauteur-2 :
        for i in range(h-1):

            sequence=[]

            for j in range(w-1):
                sequence.append((i,j))

                res = random.randint(0,1)
                if res == 0:
                    laby.remove_wall((i,j),(i,j+1))
                else:
                    cell = random.choice(sequence)
                    laby.remove_wall((cell[0], cell[1]), (cell[0]+1, cell[1]))
                    sequence = []
                
                sequence.append((i,w-1))
                cell = random.choice(sequence)
                laby.remove_wall((cell[0], cell[1]), (cell[0]+1, cell[1]))

            for k in range(0, h):
                if k+1 < h :
                    laby.remove_wall((h-1, k), (h-1, k+1))
                    
        return laby
    
    @classmethod
    def gen_fusion(cls,h,w):
        """
        génere un labirynthe via l'algorythme de fusion de chemin
        """
        laby=cls(h,w,empty=False)
        laby.fill()
        cells_lst=laby.get_cells()
        
        label_cells = {}
        label = 1

        for cell in laby.neighbors.keys():
            label_cells[cell] = label
            label += 1
        
        walls=laby.get_walls()
        random.shuffle(walls)
        for i in walls:
            if label_cells[i[0]] != label_cells[i[1]]:
                #print("wall",i," ", label_cells[i[0]]," cell2 ",label_cells[i[1]])
                laby.remove_wall(i[0],i[1])
                #print(i[0],i[1])
                tmp=label_cells[i[1]]
                for cell in cells_lst:
                    if label_cells[cell]==tmp:
                        label_cells[cell]=label_cells[i[0]]
                #print("label cell 2",label_cells[i[1]])

                
        return laby
    
    
    @classmethod
    def gen_exploration(cls,h,w):
        """
        génere un labirynthe via l'algorythme d'exploration exhaustive
        """
        
        laby=cls(h,w,empty=False)
        laby.fill()
        cells_lst=laby.get_cells()
        visit_cells = {}

        for cell in laby.neighbors.keys():
            visit_cells[cell] = False
            
            
        random_cell=random.randint(0,len(cells_lst)-1)
        
        visit_cells[cells_lst[random_cell]] = True
        
        pile=[]
        
        pile.insert(0,cells_lst[random_cell])
        while pile != []:
            elem=pile.pop(0)
            
            voisin_test=False
            for i in laby.get_contiguous_cells(elem):
                if visit_cells[i]==False:
                    voisin_test=True
            if voisin_test==True:
                pile.insert(0,elem)

                random_neighbour=random.choice(laby.get_contiguous_cells(elem))

                laby.remove_wall(elem,random_neighbour)

                visit_cells[random_neighbour]=True

                pile.insert(0,random_neighbour)

        return laby
    
    @classmethod
    def gen_wilson(cls,h,w):
        """
        génere un labirynthe via l'algorythme de wilson
        """
        laby = cls(h, w, empty=False)
        laby.fill()
        
        cells_lst=laby.get_cells()
        cell = random.choice(cells_lst)
        
        visited = []
        visited.append(cell)
        
        
        while len(visited) < len(cells_lst):
            cell = random.choice(cells_lst)
            while cell in visited:
                cell = random.choice(cells_lst)
            path = [cell]

            while path[-1] not in visited:
                moove_to_cell = random.choice(laby.get_contiguous_cells(path[-1]))

                if moove_to_cell in path:
                    path = path[:path.index(moove_to_cell)+1]
                else:
                    path.append(moove_to_cell)

            for i in range(len(path)-1):
                laby.remove_wall(path[i], path[i+1])

            visited+= path

        return laby
    
    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #    self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt   
        
    """
    Compléter manquante
    """
    def solve_dfs(self, start, stop):
        Pile = [start]
        mark = [start]
        pred = {start : start}
        
        c = ""
        
        for i in range(self.height):
            for j in range(self.width):
                if (i,j) not in mark:
                    c = (i,j)
                    if c == stop:
                        pred[c] = start
                    else:
                        for voisine in self.neighbors[c]:
                            if voisine not in mark:
                                mark.append(voisine)
                                Pile.append(voisine)
                                pred[voisine] = c
                                
                                
        chemin = []
        c = stop
        while c != start:
            chemin.append(c)
            c = pred[c]
        chemin.append(start)
        
        return chemin

In [52]:
laby = Maze.gen_fusion(15, 15)
solution = laby.solve_dfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D             ┃           ┃   ┃           ┃           ┃   ┃
┣   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃   ┃   ┃       ┃   ┃           ┃               ┃           ┃
┣   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━┫
┃   ┃   ┃       ┃   ┃   ┃   ┃   ┃       ┃   ┃   ┃   ┃       ┃
┣   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━┫
┃       ┃   ┃       ┃   ┃   ┃                           ┃   ┃
┣   ╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ┫
┃   ┃       ┃   ┃   ┃       ┃           ┃                   ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━┫
┃       ┃       ┃       ┃   ┃   ┃   ┃   ┃   ┃       ┃       ┃
┣━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━┫
┃   ┃   ┃   ┃           ┃               ┃   ┃       ┃       ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃       ┃           ┃           ┃   ┃   ┃               ┃   ┃
┣   ╋━━━